In [20]:
import re ## Exprexiones regulares
import pandas as pd
import numpy as np
import nltk ## Procesamiento de lenguaje natural
from nltk.corpus import stopwords ## Palabras vacias
from nltk.stem import SnowballStemmer ## Stemming
from nltk.tokenize import word_tokenize ## Tokenizacion
from nltk.tokenize import RegexpTokenizer ## Tokenizacion
from sklearn.feature_extraction.text import CountVectorizer ## Vectorizador
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
# import spacy ## Procesamiento de lenguaje natural

############## Descarga de recursos de nltk ################
nltk.download('punkt') ## Tokenizador
nltk.download('stopwords') ## Palabras vacias
nltk.download('snowball_data') ## Stemming
nltk.download('wordnet') ## Lematizacion

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ingen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ingen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package snowball_data to
[nltk_data]     C:\Users\ingen\AppData\Roaming\nltk_data...
[nltk_data]   Package snowball_data is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ingen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
df_corp = pd.read_csv('./data_raw/proceso_reingenieria_automatizacion_20241130_180543.csv', sep=',') ## Carga de datos
df_corp.head()

,Categoria,Contenido
0,Fundamentos BPR definicion,La reingeniería de procesos de negocio (BPR) e...
1,Fundamentos BPR principios clave,"Organización alrededor de resultados, no tarea..."
2,Fundamentos BPR beneficios,"Reducción de costos operativos, Mejora en la c..."
3,Automatizacion Procesos definicion,La automatización de procesos utiliza tecnolog...
4,Automatizacion Procesos tecnologias clave,"RPA (Robotic Process Automation), BPA (Busines..."


In [14]:
df_corp['Contenido_Normalizado'] = df_corp['Contenido'].str.lower()

# preprocesamiento de texto
def preprocesar_texto(texto):
    ## Eliminacion de caracteres especiales
    texto = re.sub(r'[^A-Za-z0-9ñáéíóúü ]', '', texto)
    ## Tokenizacion
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(texto)
    ## Eliminacion de palabras vacias
    stop_words = set(stopwords.words('spanish'))
    tokens = [i for i in tokens if i not in stop_words]
    ## Stemming
    stemmer = SnowballStemmer('spanish')
    tokens = [stemmer.stem(i) for i in tokens]
    ## Lematizacion
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(i) for i in tokens]
    return ' '.join(tokens)

In [16]:
# Aplicar preprocesamiento
df_corp['Contenido_Procesado'] = df_corp['Contenido_Normalizado'].apply(preprocesar_texto)

In [18]:
# Crear y aplicar TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=100, 
                                 stop_words='english',
                                 ngram_range=(1, 2))
tfidf_matrix = tfidf_vectorizer.fit_transform(df_corp['Contenido_Procesado'])

In [19]:
# Obtener los términos más relevantes
feature_names = tfidf_vectorizer.get_feature_names_out()
dense = tfidf_matrix.todense()
denselist = dense.tolist()
scores = pd.DataFrame(denselist, columns=feature_names)

In [21]:
# Obtener los top términos por TF-IDF
top_terms = pd.DataFrame({
    'term': feature_names,
    'score': np.array(tfidf_matrix.sum(0)).ravel()
}).sort_values('score', ascending=False).head(15)

print("\nTop 15 términos más relevantes según TF-IDF:")
print(top_terms)


Top 15 términos más relevantes según TF-IDF:
                 term     score
60             proces  2.225034
2           automatiz  1.878924
92            servici  1.402652
93                tar  1.187482
95              tiemp  1.048229
74            process  1.034839
58           prim año  1.000000
83            promedi  1.000000
0          automation  0.861722
15         estandariz  0.826745
20             manten  0.807188
96         tiemp cicl  0.774114
9                cicl  0.774114
27              mejor  0.771980
65  proces estandariz  0.712948


In [22]:
# Realizar NMF para identificar tópicos
n_topics = 5
nmf_model = NMF(n_components=n_topics, random_state=42)
nmf_output = nmf_model.fit_transform(tfidf_matrix)

In [ ]:
# Obtener los términos más importantes por tópico
def get_topics(model, feature_names, n_top_words):
    topics = []
    for topic_idx, topic in enumerate(model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]]
        topics.append(top_words)
    return topics

topics = get_topics(nmf_model, feature_names, 5)

print("\nTópicos principales identificados:")
for idx, topic in enumerate(topics):
    print(f"\nTópico {idx + 1}: {', '.join(topic)}")


Tópicos principales identificados:

Tópico 1: automation, process, process automation, automation bpa, rpa

Tópico 2: servici, mejor, proces, calid servici, calid

Tópico 3: tiemp, tiemp cicl, cicl, reduccion, product

Tópico 4: automatiz, tar, proces automatiz, empresarial, proces empresarial

Tópico 5: proces, manten, proces simpl, document, proces estandariz
